In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pennylane as qml

In [51]:
# Load data
df = pd.read_csv('./dataset.csv')

# Prepare features and target
X = df.drop(columns=['COPDSEVERITY'])
y = df['COPDSEVERITY']

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Handle missing values - fill with median for numeric columns
X = X.fillna(X.median(numeric_only=True))

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

In [25]:
y_test

array([2, 2, 0, 2, 0, 0, 2, 1, 2, 2, 3, 2, 0, 1, 1, 2, 0, 1, 3, 2, 1])

In [52]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [27]:
X_train_scaled

array([[ 1.31667736,  0.43116841, -0.38179255, ..., -0.37796447,
        -0.5       , -0.33333333],
       [-0.85836489, -0.14436228,  0.7090433 , ..., -0.37796447,
        -0.5       , -0.33333333],
       [-0.30597321, -0.98847397, -0.51814703, ..., -0.37796447,
        -0.5       ,  3.        ],
       ...,
       [-1.27265865,  1.4095706 , -0.10908358, ..., -0.37796447,
        -0.5       , -0.33333333],
       [ 1.4202508 ,  1.42875496, -0.38179255, ..., -0.37796447,
        -0.5       , -0.33333333],
       [ 0.00474712, -0.95010526,  0.0272709 , ...,  2.64575131,
        -0.5       , -0.33333333]], shape=(80, 23))

In [53]:

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.LongTensor(y_test)


In [54]:
# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [55]:
input_dim = X_train.shape[1]
hidden1_dim = 23
n_qubits = 4  # Number of qubits for PQC
output_dim = len(np.unique(y_encoded))

In [56]:
dev = qml.device("default.qubit", wires=n_qubits)

In [57]:
@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs, weights):
    """
    Parameterized Quantum Circuit
    Args:
        inputs: classical input data (reduced dimension)
        weights: trainable quantum parameters
    """
    # Encode classical data into quantum state
    for i in range(n_qubits):
        qml.RY(inputs[i % len(inputs)], wires=i)
    
    # Parameterized quantum layers
    n_layers = weights.shape[0]
    for layer in range(n_layers):
        # Rotation gates with trainable parameters
        for i in range(n_qubits):
            qml.RY(weights[layer, i, 0], wires=i)
            qml.RZ(weights[layer, i, 1], wires=i)
        
        # Entangling gates
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
        qml.CNOT(wires=[n_qubits - 1, 0])  # Circular entanglement
    
    # Measure expectations
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [62]:
class QuantumNeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden1_dim, n_qubits, output_dim, n_qlayers=2):
        super(QuantumNeuralNetwork, self).__init__()
        
        # Classical layers
        self.fc1 = nn.Linear(input_dim, hidden1_dim)
        self.fc2 = nn.Linear(hidden1_dim, n_qubits)  # Reduce to qubit dimension
        
        # Quantum layer parameters
        self.n_qubits = n_qubits
        self.n_qlayers = n_qlayers
        self.q_params = nn.Parameter(torch.randn(n_qlayers, n_qubits, 2) * 0.1)
        
        # Classical output layer
        self.fc3 = nn.Linear(n_qubits, output_dim)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.tanh = nn.Tanh()  # For quantum input encoding
        
    def forward(self, x):
        # Classical preprocessing
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.tanh(self.fc2(x))  # Normalize to [-1, 1] for quantum encoding
        
        # Quantum layer
        # Process each sample in the batch
        q_out_list = []
        for sample in x:
            q_out = quantum_circuit(sample, self.q_params)
            q_out_list.append(torch.stack(q_out))
        
        q_out_batch = torch.stack(q_out_list).float()
        
        # Classical output layer
        output = self.fc3(q_out_batch)
        
        return output

In [63]:
# Initialize model
model = QuantumNeuralNetwork(input_dim, hidden1_dim, n_qubits, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:

print(f'Model Architecture:')
print(f'Input Layer: {input_dim} neurons')
print(f'Hidden Layer 1 (Classical): {hidden1_dim} neurons')
print(f'Hidden Layer 2 (Quantum): {n_qubits} qubits')
print(f'Output Layer: {output_dim} neurons')
print(f'\nQuantum circuit has {model.n_qlayers} parameterized layers')
print(f'Total quantum parameters: {model.q_params.numel()}\n')

In [ ]:
num_epochs = 250  # Reduced due to quantum circuit overhead
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if (epoch + 1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}')

In [72]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    _, predicted = torch.max(y_pred, 1)
    acc = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
    print(f'\nAccuracy: {acc:.4f}')


Accuracy: 0.9048


In [ ]:

print(f'\nModel Architecture:')
print(f'Input Layer: {input_dim} neurons')
print(f'Hidden Layer 1: {hidden1_dim} neurons')
print(f'Hidden Layer 2: {hidden2_dim} neurons')
print(f'Output Layer: {output_dim} neurons')

In [73]:

# Save the model
save_dict = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scaler': scaler,
    'label_encoder': label_encoder,
    'input_dim': input_dim,
    'hidden1_dim': hidden1_dim,
    'n_qubits': n_qubits,
    'output_dim': output_dim,
    'n_qlayers': model.n_qlayers,
    'accuracy': acc
}

torch.save(save_dict, 'quantum_copd_model.pth')
print('\nModel saved to: quantum_copd_model.pth')

# Optional: Visualize quantum circuit
print(f'\nQuantum Circuit Structure:')
print(qml.draw(quantum_circuit)(torch.randn(n_qubits), model.q_params))


Model saved to: quantum_copd_model.pth

Quantum Circuit Structure:
0: ──RY(0.57)───RY(0.70)──RZ(1.07)──╭●───────╭X──RY(-0.42)──RZ(-0.04)─╭●───────╭X─┤  <Z>
1: ──RY(1.85)───RY(0.01)──RZ(1.27)──╰X─╭●────│───RY(0.39)───RZ(-0.05)─╰X─╭●────│──┤  <Z>
2: ──RY(1.47)───RY(0.44)──RZ(-1.04)────╰X─╭●─│───RY(-0.21)──RZ(-0.15)────╰X─╭●─│──┤  <Z>
3: ──RY(-0.06)──RY(0.74)──RZ(0.84)────────╰X─╰●──RY(-0.18)──RZ(-0.05)───────╰X─╰●─┤  <Z>
